# Model 2 Version 2 - Creating a Ranked List using a given playlist title and commonly shared Songs.

In [27]:
#Imports all necessary dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlite3
import json

def id_from_uri(uri: str):
    """Helper method to get the ID from a URI string like so:
    URI: 'spotify:artist:012345...'
    ID: '012345...'
    """
    return uri.split(':')[2]
def get_songs_from_playlist_slice(connection: sqlite3.Connection, playlistIDs):
    """Get the list of songs (IDs) from a list of playlist IDS.
    """
    songQuery = "SELECT songID FROM songs WHERE playlistID = ?"
    foundSongIDs = [] #List containing the songs found in the playlists
    for playlist in playlistIDs:
        songQuery = f"SELECT songID FROM songs WHERE playlistID = '{playlist}'"
        desiredSongIDs = pd.read_sql(songQuery, song_conn)
        foundSongIDs.append(desiredSongIDs.values.flatten().tolist()) #converts from pandas dataframe to a list
    return foundSongIDs
    
'''
#Code that uses JSON slices
      
#Intializes file name data variables
slice_lower = 0
slice_upper = 999
listOfSongIDs = list() #List that contains the list of song ids associated with the playlists
while(slice_upper <= 999999):
    fileName = f"../../data/playlist/mpd.slice.{slice_lower}-{slice_upper}.json" #Creates the file name for the json file we are analyzing
    #Opens the json file and load the slice into a data variable
    with open(fileName) as testSlice:
        slice_json = json.load(testSlice)
    listOfSongIDs.extend(get_songs_from_playlist_slice(slice_json, playListName="Top Hits")) #Gets list of Song IDs from the playlist
    slice_lower += 1000 #Increments lower bound of slice to update file name
    slice_upper += 1000 #Increments upper bound of slice to update the file name
print(len(listOfSongIDs))
'''

#Code that uses playlist_song.db
#Forms a connection with the database
song_conn = sqlite3.Connection("../../data/song/playlist_song.db")
songCur = song_conn.cursor()
#Query for getting playlist IDs TO BE REMOVED
playlistQuery = "SELECT playlistID FROM songs WHERE LOWER(playlistName) = LOWER('Top Hits')"
#Gets desired playlist ID's list
desiredPlaylistsID = pd.read_sql(playlistQuery,song_conn)
desiredPlaylistsID = desiredPlaylistsID.values.flatten().tolist() #converts from pandas dataframe to a list
desiredPlaylistsID = list(set(desiredPlaylistsID)) #Gets rid of repeated values
listOfSongIDs = get_songs_from_playlist_slice(song_conn, desiredPlaylistsID) # Retrieves a list of song ID's from the given playlist IDs
listOfSongIDs.extend(listOfSongIDs)
print(len(listOfSongIDs))

[['4Hf7WnR761jpxPr5D46Bcd', '7lGKEWMXVWWTt3X71Bv44I', '6DNtNfH8hXkqOX1sjqmI7p', '4pdPtRcBmOSQDlJ3Fk945m', '68EMU2RD1ECNeOeJ5qAXCV', '6875MeXyCW0wLyT72Eetmo', '11KJSRSgaDxqydKYiD2Jew', '7yHEDfrJNd0zWOfXwydNH0', '09rmUwnx6muhjV7PvhaZU8', '0QsvXIfqM0zZoerQfsI9lm', '2XMTqoHHSH0lvuXrvIEdco', '0azC730Exh71aQlOt9Zj3y', '7nD9nN3jord9wWcfW3Gkcm', '4qqArAiTPueDxIp7cf87h7', '4TZy1wLyHec06pwgFYDh1a', '27SdWb2rFzO6GWiYDBTD9j', '0prNGof3XqfTvNDxHonvdK', '638fKB27P7UAmu3Aq0vEJ3', '7vRriwrloYVaoAe3a9wJHe', '0OwX5aROoW1Iip8FV51Efg', '0i7bg5CcqyaceKjXaPXywN', '7KxhSJOYiqCDclXDBNlFSZ', '46jLy47W8rkf8rEX04gMKB', '1Tt4sE4pXi57mTD1GCzsqm', '16Ah4QqH4mgYVXqfC4mdSd', '12REd1n8PeiHHWOh066tpr', '27PmvZoffODNFW2p7ehZTQ', '0CNjoQozZP0mB9rKVdrKGx', '4tCtwWceOPWzenK2HAIJSb', '2DpCdPMg1BADE4HDnxt3Rd', '4esOae7i4rqTbAu9o5Pxco', '0HMjXBAZmSYOTTi33WpMso', '3NLnwwAQbbFKcEcV8hDItk', '13HVjjWUZFaWilh2QUJKsP', '1tYt8PbpbeTuqsNmprAZYY', '7BKLCZ1jbUBVqRi2FVlTVw', '5FljCWR0cys07PQ9277GTz', '3pzjHKrQSvXGHQ98dx18HI', '4pLwZjInH

Now that we have obtained the list of Song IDS from the relevant playlist. We now create a ranked list with the top K song ID's relevant to the playlist. We will be ranking the list in terms of the songs appearance in these playlists.

In [22]:
songDict = {} #Dictionary that contains the number of occurences for each song.
#Goes through the list of song ids and keeps track the number of occurences for each song.
for songID in listOfSongIDs:
    if songID in songDict.keys():
        songDict[songID] = songDict[songID] + 1 #increments the song ID occurence counter by 1 if the song id has been found
    else:
        songDict[songID] = 1 #Intializes the new key and value for the new song id.
#Gets the contents of the dictionary and formulates it into a list of tuples.
rankedsongsList = list(songDict.items())
#Sorts the song id list by the most occurences to the least amount of occurences.
rankedsongsList = sorted(rankedsongsList, key=lambda occurence: occurence[1], reverse=True)
print(rankedsongsList)

TypeError: unhashable type: 'list'